In [45]:
import os;
import sys;
import numpy as np;
from sklearn.linear_model import LinearRegression;

In [46]:
def eng_read_sudha_format( fname):
    model = dict();
    f = open( fname, 'r');
    lcnt = 0;
    for line in f:
        if lcnt == 0:
            lcnt += 1;
            continue;
        tokens = line.strip().split( ' ');
        key = tokens[ 0];
        val = np.asarray( [ float( token) for token in tokens[ 1:]]);
        word, pos, sid = key.split( '@');
        if pos == 'noun':
            pos = 'n';
        if pos == 'adjective':
            pos = 'a';
        if pos == 'verb':
            pos = 'v';
        if pos == 'adverb':
            pos = 'r';
        mkey = sid + '_' + pos;
        model[ mkey] = val;
    return model;

In [47]:
def hin_read_sudha_format( fname):
    model = dict();
    f = open( fname, 'r');
    lcnt = 0;
    for line in f:
        if lcnt == 0:
            lcnt += 1;
            continue;
        tokens = line.strip().split( ' ');
        key = tokens[ 0];
        val = np.asarray( [ float( token) for token in tokens[ 1:]]);
        word, sid = key.split( '@');
        mkey = sid;
        model[ mkey] = val;
    return model;

In [48]:
def add_pos_to_hin( model):
    ret_model = dict();
    adjs = [];
    advs = [];
    nouns = [];
    verbs = [];
    f = open( fprefix + 'adj.idx', 'r');
    for line in f:
        adjs.append( line.strip());
    f.close();
    f = open( fprefix + 'adv.idx', 'r');
    for line in f:
        advs.append( line.strip());
    f.close();
    f = open( fprefix + 'noun.idx', 'r');
    for line in f:
        nouns.append( line.strip());
    f.close();
    f = open( fprefix + 'verb.idx', 'r');
    for line in f:
        verbs.append( line.strip());
    f.close();
    for key in model:
        if key in adjs:
            mkey = key + '_a';
        if key in advs:
            mkey = key + '_r';
        if key in nouns:
            mkey = key + '_n';
        if key in verbs:
            mkey = key + '_v';
        ret_model[ mkey] = model[ key];
    return ret_model;

In [49]:
fprefix = '/home/development/kevin/workspace/linking/data/';
fname = fprefix + 'english-sense-vector_sb.txt';
eng_m = eng_read_sudha_format( fname);

In [50]:
to_remove = [];
for key in eng_m:
    val = eng_m[ key];
    if np.linalg.norm( val) < 1e-6:
        to_remove.append( key);
for elem in to_remove:
    del eng_m[ elem];

In [51]:
print( type( eng_m[ '2430830_n']));
print( len( eng_m[ '2430830_n']));
print( np.shape( eng_m[ '2430830_n']));


<class 'numpy.ndarray'>
300
(300,)


In [53]:
#fname = fprefix + 'hindi-sense-vector_sb.txt';
fname = fprefix + 'hindi_cbow_300_5_10_sense.txt';
hin_m = hin_read_sudha_format( fname);
hin_m = add_pos_to_hin( hin_m);

In [54]:
print( type( hin_m[ '16074_n']));
print( len( hin_m[ '16074_n']));
print( np.shape( hin_m[ '16074_n']));

<class 'numpy.ndarray'>
300
(300,)


In [55]:
hin_dict = dict();
eng_dict = dict();
pos_list = [ 'a', 'n', 'r', 'v'];
for pos in pos_list:
    hin_dict[ pos] = dict();
    eng_dict[ pos] = dict();

In [56]:
for key in hin_m:
    if 'a' in key:
        hin_dict[ 'a'][ key] = hin_m[ key];
    if 'n' in key:
        hin_dict[ 'n'][ key] = hin_m[ key];
    if 'r' in key:
        hin_dict[ 'r'][ key] = hin_m[ key];
    if 'v' in key:
        hin_dict[ 'v'][ key] = hin_m[ key];

for key in eng_m:
    if 'a' in key:
        eng_dict[ 'a'][ key] = eng_m[ key];
    if 'n' in key:
        eng_dict[ 'n'][ key] = eng_m[ key];
    if 'r' in key:
        eng_dict[ 'r'][ key] = eng_m[ key];
    if 'v' in key:
        eng_dict[ 'v'][ key] = eng_m[ key];

In [57]:
hlist = dict();
elist = dict();
link_list = [];
fname = fprefix + 'links.csv';
f = open( fname, 'r');
mcnt = 0; # number of direct links which are mismatch in pos
tcnt = 0;
cnt_dict = dict();
for line in f:
    tokens = line.strip().split( ',');
    if tokens[ 1] == 'ADJECTIVE':
        p1 = 'a';
    if tokens[ 1] == 'NOUN':
        p1 = 'n';
    if tokens[ 1] == 'VERB':
        p1 = 'v';
    if tokens[ 1] == 'ADVERB':
        p1 == 'r';
    if tokens[ 3] == 'ADJECTIVE':
        p2 = 'a';
    if tokens[ 3] == 'NOUN':
        p2 = 'n';
    if tokens[ 3] == 'VERB':
        p2 = 'v';
    if tokens[ 3] == 'ADVERB':
        p2 == 'r';
    k1 = tokens[ 0] + '_' + p1;
    k2 = tokens[ 2] + '_' + p2;
    if p1 != p2:
        mcnt += 1;
    else:
        if p1 not in cnt_dict:
            cnt_dict[ p1] = 0;
        cnt_dict[ p1] += 1
        link_list.append( ( k1, k2, p1));
    tcnt += 1;

In [58]:
print( mcnt);
print( cnt_dict[ 'a']);
print( cnt_dict[ 'n']);
print( cnt_dict[ 'v']);
print( tcnt);

11
3602
11709
2003
17325


In [59]:
print( len( link_list));

17314


In [60]:
avail_list = [];
avail_dict = dict();
for tup in link_list:
    k1, k2, pos = tup;
    if pos not in avail_dict:
        avail_dict[ pos] = [];
    if k1 not in hin_m:
        continue;
    if k2 not in eng_m:
        continue;
    avail_list.append( tup);
    avail_dict[ pos].append( tup);
print( len( avail_list));
for key in avail_dict:
    print( len( avail_dict[ key]));

8398
0
7064
1334


In [61]:
avail_dict[ 'r'] = [];
print( avail_dict.keys());
print( eng_dict.keys());
print( hin_dict.keys());
for key in avail_dict:
    print( key, len( avail_dict[ key]));

dict_keys(['a', 'r', 'n', 'v'])
dict_keys(['a', 'r', 'n', 'v'])
dict_keys(['a', 'r', 'n', 'v'])
a 0
r 0
n 7064
v 1334


In [62]:
def train_h2e( train_list, inp_m, out_m, bias_p = False):
    ivlist = [];
    ovlist = [];
    for tup in train_list:
        k1, k2, pos = tup;
        ivlist.append( inp_m[ k1]);
        ovlist.append( out_m[ k2]);
    i_data = np.asarray( ivlist, dtype = 'f4');
    o_data = np.asarray( ovlist, dtype = 'f4');
    dim = len( o_data[ 0]);
    #print( dim);
    dmodel_list = [];
    print( 'Training', flush = True);
    for i in range( dim):
        #print( 'Dim = ', i, flush = True);
        X = i_data;
        Y = o_data[ :, i];
        cur_model = LinearRegression( fit_intercept = bias_p);
        cur_model.fit( X, Y);
        dmodel_list.append( cur_model);
    return dmodel_list;

In [63]:
def apply_model( test_list, inp_m, out_m, model_list):
    ivlist = [];
    ovlist = [];
    for tup in test_list:
        k1, k2, pos = tup;
        ivlist.append( inp_m[ k1]);
        ovlist.append( out_m[ k2]);
    i_data = np.array( ivlist, dtype = 'f4');
    dim = len( ovlist[ 0]);
    ypred = [];
    print( 'Translating', flush = True);
    for i in range( dim):
        #print( 'Dim = ', i, flush = True);
        X = i_data;
        cur_model = model_list[ i];
        Yp = cur_model.predict( X);
        ypred.append( Yp);
    pred_data = ypred[ 0];
    for i in range( 1, dim):
        pred_data = np.column_stack( ( pred_data, ypred[ i]));
    #print( np.shape( pred_data));
    return pred_data;

In [64]:
def get_acc( test_list, inp_m, out_m, pred_data):
    print( 'Sim Scoring', flush = True);
    ivlist = [];
    ovlist = [];
    ptr = 0;
    k2id = dict();
    id2k = dict();
    for key in out_m:
        ovlist.append( out_m[ key]);
        k2id[ key] = ptr;
        id2k[ ptr] = key;
        ptr += 1;
    o_data = np.asarray( ovlist, dtype = 'f4');
    no_data = ( o_data.T / np.linalg.norm( o_data, axis = 1)).T;
    npred_data = ( pred_data.T / np.linalg.norm( pred_data, axis = 1)).T;
    sim_mat = npred_data.dot( no_data.T);
    r, c = np.shape( sim_mat);
    #trim_sim_mat = np.sort( sim_mat)[ :, ::-1];
    trim_ind_mat = np.argsort( sim_mat)[ :, ::-1];
    indx = [];
    posl = [];
    print( 'Ranking', flush = True);
    for i in range( r):
        true = test_list[ i][ 1];
        pos = test_list[ i][ 2];
        tlist = trim_ind_mat[ i].tolist();
        tind = tlist.index( k2id[ true]);
        indx.append( tind);
        posl.append( pos);
    acc_01 = sum( [ elem < 1 for elem in indx]);
    acc_03 = sum( [ elem < 3 for elem in indx]);
    acc_05 = sum( [ elem < 5 for elem in indx]);
    acc_08 = sum( [ elem < 8 for elem in indx]);
    acc_10 = sum( [ elem < 10 for elem in indx]);
    acc_20 = sum( [ elem < 20 for elem in indx]);
    acc_50 = sum( [ elem < 50 for elem in indx]);
    acc_100 = sum( [ elem < 100 for elem in indx]);
    print( acc_01 / len( indx));
    print( acc_03 / len( indx));
    print( acc_05 / len( indx));
    print( acc_08 / len( indx));
    print( acc_10 / len( indx));
    print( acc_20 / len( indx));
    print( acc_50 / len( indx));
    print( acc_100 / len( indx));
    print( min( indx), max( indx), len( indx));

In [65]:
def cross_validation( link_list, inp_m, out_m, bias_p = False):
    fold_beg = [ 0] * 10;
    fold_end = [ 0] * 10;
    for i in range( 1, 10):
        fold_end[ i - 1] = (len( link_list) // 10) * i;
        fold_beg[ i] = (len( link_list) // 10) * i;
    fold_end[ 9] = len( link_list);
    for fold_ptr in range( 10):
        print( 'Running fold ', fold_ptr, flush = True);
        train_list = [];
        test_list = [];
        for i in range( 10):
            if i != fold_ptr:
                train_list.extend( link_list[ fold_beg[ i] : fold_end[ i]]);
            else:
                test_list.extend( link_list[ fold_beg[ i] : fold_end[ i]]);
        dmodel_list = train_h2e( train_list, inp_m, out_m, bias_p);
        pred_data = apply_model( test_list, inp_m, out_m, dmodel_list);
        get_acc( test_list, inp_m, out_m, pred_data);

In [66]:
cross_validation( avail_dict[ 'v'], hin_dict[ 'v'], eng_dict[ 'v']);

Running fold  0
Training
Translating
Sim Scoring
Ranking
0.022556390977443608
0.045112781954887216
0.06766917293233082
0.09022556390977443
0.16541353383458646
0.21052631578947367
0.3157894736842105
0.3684210526315789
0 11255 133
Running fold  1
Training
Translating
Sim Scoring
Ranking
0.045112781954887216
0.08270676691729323
0.11278195488721804
0.17293233082706766
0.20300751879699247
0.2781954887218045
0.3458646616541353
0.38345864661654133
0 8024 133
Running fold  2
Training
Translating
Sim Scoring
Ranking
0.045112781954887216
0.13533834586466165
0.17293233082706766
0.21804511278195488
0.23308270676691728
0.2781954887218045
0.3458646616541353
0.42857142857142855
0 9805 133
Running fold  3
Training
Translating
Sim Scoring
Ranking
0.07518796992481203
0.11278195488721804
0.12781954887218044
0.17293233082706766
0.18796992481203006
0.2631578947368421
0.3609022556390977
0.41353383458646614
0 11156 133
Running fold  4
Training
Translating
Sim Scoring
Ranking
0.06015037593984962
0.13533834586

In [67]:
cross_validation( avail_dict[ 'n'], hin_dict[ 'n'], eng_dict[ 'n']);

Running fold  0
Training
Translating
Sim Scoring
Ranking
0.13739376770538245
0.2577903682719547
0.3314447592067989
0.38526912181303113
0.40368271954674223
0.4985835694050991
0.5694050991501416
0.6444759206798867
0 24950 706
Running fold  1
Training
Translating
Sim Scoring
Ranking
0.12889518413597734
0.24504249291784702
0.311614730878187
0.3640226628895184
0.3838526912181303
0.47592067988668557
0.5524079320113314
0.6288951841359773
0 25960 706
Running fold  2
Training
Translating
Sim Scoring
Ranking
0.08923512747875353
0.17847025495750707
0.23654390934844194
0.28753541076487255
0.3101983002832861
0.3881019830028329
0.4971671388101983
0.5821529745042493
0 26198 706
Running fold  3
Training
Translating
Sim Scoring
Ranking
0.09490084985835694
0.21246458923512748
0.29036827195467424
0.3555240793201133
0.37677053824362605
0.4546742209631728
0.5354107648725213
0.6189801699716714
0 27737 706
Running fold  4
Training
Translating
Sim Scoring
Ranking
0.1388101983002833
0.2563739376770538
0.313031

In [68]:
def h2e( train_list, test_list, bias_p = False):
    evlist = [];
    hvlist = [];
    for tup in train_list:
        k1, k2, pos = tup;
        hvlist.append( hin_m[ k1]);
        evlist.append( eng_m[ k2]);
    h_data = np.array( hvlist, dtype = 'f4');
    e_data = np.array( evlist, dtype = 'f4');
    dim = len( e_data[ 0]);
    print( dim);
    dmodel_list = [];
    print( 'Training', flush = True);
    for i in range( dim):
        print( 'Dim = ', i, flush = True);
        X = h_data;
        Y = e_data[ :, i];
        cur_model = LinearRegression( fit_intercept = bias_p);
        cur_model.fit( X, Y);
        dmodel_list.append( cur_model);
    evlist = [];
    hvlist = [];
    for tup in test_list:
        k1, k2, pos = tup;
        hvlist.append( hin_m[ k1]);
        evlist.append( eng_m[ k2]);
    inp_data = np.array( hvlist, dtype = 'f4');
    
    ypred = [];
    print( 'Translating', flush = True);
    for i in range( dim):
        print( 'Dim = ', i, flush = True);
        X = inp_data;
        cur_model = dmodel_list[ i];
        Yp = cur_model.predict( X);
        ypred.append( Yp);
    trans_syn_data = ypred[ 0];
    for i in range( 1, dim):
        trans_syn_data = np.column_stack( ( trans_syn_data, ypred[ i]));
    print( np.shape( trans_syn_data));
    evlist = [];
    hvlist = [];
    ptr = 0;
    k2id = dict();
    id2k = dict();
    for key in eng_m:
        evlist.append( eng_m[ key]);
        k2id[ key] = ptr;
        id2k[ ptr] = key;
        ptr += 1;
    e_data = np.asarray( evlist, dtype = 'f4');
    print( np.shape( e_data));
    
    ntrans_syn_data = ( trans_syn_data.T / np.linalg.norm( trans_syn_data, axis = 1)).T;
    ne_data = ( e_data.T / np.linalg.norm( e_data, axis = 1)).T;
    print( np.shape( trans_syn_data));
    print( np.shape( ntrans_syn_data));
    print( np.shape( e_data));
    print( np.shape( ne_data));
    
    sim_mat = ntrans_syn_data.dot( ne_data.T);
    print( np.shape( sim_mat));
    r, c = np.shape( sim_mat);
    trim_sim_mat = np.sort( sim_mat)[ :, ::-1];
    trim_ind_mat = np.argsort( sim_mat)[ :, ::-1];
    
    print( np.shape( trim_sim_mat));
    print( np.shape( trim_ind_mat));
    
    indx = [];
    posl = [];
    for i in range( r):
        true = test_list[ i][ 1];
        pos = test_list[ i][ 2];
        tlist = trim_ind_mat[ i].tolist();
        cur_pos = 0;
        pos_pos = 0;
        for item in tlist:
            if k2id[ true] == item:
                break
            if pos in id2k[ item]:
                pos_pos += 1;
        #tind = tlist.index( k2id[ true]);
        indx.append( pos_pos);
        posl.append( pos);
    

In [69]:
#trans_syn_data = h2e( avail_list[ : 841 * 9], avail_list[ 841 * 9:]);
#trans_syn_data = h2e( avail_list[ : 841 * 9], avail_list[ 841 * 9:], bias_p = True);
#trans_syn_data = h2e( avail_list, avail_list);
trans_syn_data = h2e( avail_list, avail_list, bias_p = True);

300
Training
Dim =  0
Dim =  1
Dim =  2
Dim =  3
Dim =  4
Dim =  5
Dim =  6
Dim =  7
Dim =  8
Dim =  9
Dim =  10
Dim =  11
Dim =  12
Dim =  13
Dim =  14
Dim =  15
Dim =  16
Dim =  17
Dim =  18
Dim =  19
Dim =  20
Dim =  21
Dim =  22
Dim =  23
Dim =  24
Dim =  25
Dim =  26
Dim =  27
Dim =  28
Dim =  29
Dim =  30
Dim =  31
Dim =  32
Dim =  33
Dim =  34
Dim =  35
Dim =  36
Dim =  37
Dim =  38
Dim =  39
Dim =  40
Dim =  41
Dim =  42
Dim =  43
Dim =  44
Dim =  45
Dim =  46
Dim =  47
Dim =  48
Dim =  49
Dim =  50
Dim =  51
Dim =  52
Dim =  53
Dim =  54
Dim =  55
Dim =  56
Dim =  57
Dim =  58
Dim =  59
Dim =  60
Dim =  61
Dim =  62
Dim =  63
Dim =  64
Dim =  65
Dim =  66
Dim =  67
Dim =  68
Dim =  69
Dim =  70
Dim =  71
Dim =  72
Dim =  73
Dim =  74
Dim =  75
Dim =  76
Dim =  77
Dim =  78
Dim =  79
Dim =  80
Dim =  81
Dim =  82
Dim =  83
Dim =  84
Dim =  85
Dim =  86
Dim =  87
Dim =  88
Dim =  89
Dim =  90
Dim =  91
Dim =  92
Dim =  93
Dim =  94
Dim =  95
Dim =  96
Dim =  97
Dim =  98
Dim =  

In [70]:
import pickle;

print( 'Ranking', flush = True);

test_list = avail_list;
#test_list = avail_list[ 841 * 9:];


print( np.shape( trans_syn_data));
evlist = [];
hvlist = [];
ptr = 0;
k2id = dict();
id2k = dict();
for key in eng_m:
    evlist.append( eng_m[ key]);
    k2id[ key] = ptr;
    id2k[ ptr] = key;
    ptr += 1;
e_data = np.asarray( evlist, dtype = 'f4');
print( np.shape( e_data));

ntrans_syn_data = ( trans_syn_data.T / np.linalg.norm( trans_syn_data, axis = 1)).T;
ne_data = ( e_data.T / np.linalg.norm( e_data, axis = 1)).T;
print( np.shape( trans_syn_data));
print( np.shape( ntrans_syn_data));
print( np.shape( e_data));
print( np.shape( ne_data));

sim_mat = ntrans_syn_data.dot( ne_data.T);


Ranking
()
(48358, 300)


AttributeError: 'NoneType' object has no attribute 'T'

In [ ]:
print( np.shape( sim_mat));

In [ ]:
r, c = np.shape( sim_mat);

In [ ]:
trim_sim_mat = np.sort( sim_mat)[ :, ::-1]
trim_ind_mat = np.argsort( sim_mat)[ :, ::-1]

In [ ]:
print( np.shape( trim_sim_mat));
print( np.shape( trim_ind_mat));

In [ ]:
indx = [];
posl = [];
for i in range( r):
    true = test_list[ i][ 1];
    pos = test_list[ i][ 2];
    tlist = trim_ind_mat[ i].tolist();
    cur_pos = 0;
    pos_pos = 0;
    for item in tlist:
        if k2id[ true] == item:
            break
        if pos in id2k[ item]:
            pos_pos += 1;
    #tind = tlist.index( k2id[ true]);
    indx.append( pos_pos);
    posl.append( pos);

In [ ]:
acc_01 = sum( [ elem < 1 for elem in indx]);
acc_03 = sum( [ elem < 3 for elem in indx]);
acc_05 = sum( [ elem < 5 for elem in indx]);
acc_08 = sum( [ elem < 8 for elem in indx]);
acc_10 = sum( [ elem < 10 for elem in indx]);
acc_20 = sum( [ elem < 20 for elem in indx]);
acc_50 = sum( [ elem < 50 for elem in indx]);
acc_100 = sum( [ elem < 100 for elem in indx]);

In [ ]:
print( acc_01 / len( avail_list));
print( acc_03 / len( avail_list));
print( acc_05 / len( avail_list));
print( acc_08 / len( avail_list));
print( acc_10 / len( avail_list));
print( acc_20 / len( avail_list));
print( acc_50 / len( avail_list));
print( acc_100 / len( avail_list));
print( min( indx), max( indx), len( indx));

In [159]:
print( test_list[ : 10]);
print( trim_ind_mat[ : 10]);

[('33093', '14377617_n', 'n'), ('33140', '9862345_n', 'n'), ('33154', '2964634_n', 'n'), ('33269', '839778_n', 'n'), ('33273', '331102_n', 'n'), ('33508', '3956157_n', 'n'), ('33550', '13667643_n', 'n'), ('33595', '14941407_n', 'n'), ('33664', '11502102_n', 'n'), ('33684', '1067070_n', 'n')]
[[46331 18950 20428 ..., 43673 38788 36346]
 [48396 16127 16136 ..., 32260 32259     0]
 [48396 16127 16136 ..., 32260 32259     0]
 ..., 
 [48396 16127 16136 ..., 32260 32259     0]
 [48396 16127 16136 ..., 32260 32259     0]
 [19661 16982 21814 ..., 13461 12002 22947]]


In [148]:
print( np.sort( tp)[ :, ::-1]);
print( np.argsort( tp)[ :, ::-1]);

[[3 2 1]
 [3 2 1]
 [3 2 1]]
[[1 2 0]
 [1 0 2]
 [0 1 2]]


In [147]:
x = np.sort( tp)[ :,:: -1];
print( x) ;

[[3 2 1]
 [3 2 1]
 [3 2 1]]


In [19]:
zecnt = 0;
for key in eng_m:
    val = eng_m[ key];
    if np.linalg.norm( val) < 1e-6:
        zecnt += 1;
print( zecnt);

39


In [22]:
zhcnt = 0;
for key in hin_m:
    val = hin_m[ key];
    if np.linalg.norm( val) < 1e-15:
        zhcnt += 1;
print( zhcnt);

0
